## Preprocessing Goal
1. Load data, group the original training and testing dataset together
2. Basic statistics
3. Transform to dataframe structure:

---
- each raw represents a single frame
- each column represent its associated information
    - video
    - frame
    - ID:video name_frame ID
    - label
    - resident_x_nose
    - resident_y_nose
    - ...
    - resident_x_tail
    - resident_y_tail
 

In [2]:
#import neccessary packages
import numpy as np
import pandas as pd

In [3]:
# @title Download and unzip the data
import os, requests, zipfile

fname = 'task1_classic_classification.zip'
url = "https://data.caltech.edu/records/s0vdx-0k302/files/task1_classic_classification.zip?download=1"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)
else:
  print('Data have already been downloaded!!!')

if not os.path.exists('task1_classic_classification'):
  # Unzip the file
  with zipfile.ZipFile(fname, 'r') as zip_ref:
    zip_ref.extractall('.')


# Download the script
fname = 'calms21_convert_to_npy.py'
url = "https://data.caltech.edu/records/s0vdx-0k302/files/calms21_convert_to_npy.py?download=1"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)

In [12]:
def load_task1_data(data_path):
  """
  Load data for task 1:
      The vocaubulary tells you how to map behavior names to class ids;
      it is the same for all sequences in this dataset.
  """
  data_dict = np.load(data_path, allow_pickle=True).item()
  dataset = data_dict['annotator-id_0']
  # Get any sequence key.
  sequence_id = list(data_dict['annotator-id_0'].keys())[0]
  vocabulary = data_dict['annotator-id_0'][sequence_id]['metadata']['vocab']
  return dataset, vocabulary


training_data, vocab = load_task1_data('data/calms21_task1_train.npy') #check where you created the files in the loading notebook
test_data, _ = load_task1_data('data/calms21_task1_test.npy') #check where you created the files in the loading notebook


TypeError: 'dict' object cannot be interpreted as an integer

In [13]:
#group training and test data together to assemble our own data
concatenated_data = {}
concatenated_data.update(training_data)
concatenated_data.update(test_data)

print(f"Number of sequences in training data: {len(training_data)}")
print(f"Number of sequences in test data: {len(test_data)}")
print(f"Number of sequences in concatenated data: {len(concatenated_data)}")

Number of sequences in training data: 70
Number of sequences in test data: 19
Number of sequences in concatenated data: 89


In [21]:
# simplify data in a dataframe

def transform_dataset(dataset):

  sequence_names = list(dataset.keys())

  data = []

  #columns
  mice = ['resident','intruder']
  coordinates = ['x', 'y']
  bodyparts = ['nose', 'left_ear', 'right_ear', 'neck', 'left_hip', 'right_hip', 'tail_base']

  print('We have ', len(sequence_names), ' sequences')

  for sequence in sequence_names:

      for f, frame in enumerate(dataset[sequence]['keypoints']):

        tabdata = {}
        id = sequence + str(f)
        tabdata = {'sequence': sequence, 'frame': f, 'id': id}
        tabdata['label'] = dataset[sequence]['annotations'][f]

        # create a column for each mouse + coordinate + bodypart column
        for m, mouse in enumerate(frame):

          for c, coordinate in enumerate(mouse):

            for b, c_bodypart in enumerate(coordinate):
              column_name = mice[m] + '_' + coordinates[c] + '_' + bodyparts[b]
              tabdata[column_name] = c_bodypart

        data.append(tabdata)

  print('We have ', len(data), ' frames in total in the dataset')
  dataset_new = data

  return dataset_new

In [1]:
whole_data = transform_dataset(concatenated_data)
df = pd.DataFrame(whole_data)
df

NameError: name 'transform_dataset' is not defined

In [24]:
# download the dataset to be reused
df.to_csv('calms21_task_data.csv',header=True)